In [21]:
# Imports
from nltk.corpus import wordnet
from nltk.tokenize import wordpunct_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk import download as nltk_download

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split

import re

import contractions

from pickle import dump

import pandas as pd
import numpy as np

In [22]:
# Global variables, constants and some other pre-requisites
# Lemmatizer as singleton, set seed for reproducibility
lemmatizer = WordNetLemmatizer()
seed = 42
dataset_path = "../en-fr.csv"

# Downloading necessary nltk data
nltk_download('averaged_perceptron_tagger')
nltk_download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/atakan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/atakan/nltk_data...


True

In [23]:
df = pd.read_csv(dataset_path, nrows=100_000)
df.head(10)

,en,fr
0,Changing Lives | Changing Society | How It Wor...,Il a transformé notre vie | Il a transformé la...
1,Site map,Plan du site
2,Feedback,Rétroaction
3,Credits,Crédits
4,Français,English
5,What is light ?,Qu’est-ce que la lumière?
6,The white light spectrum Codes in the light Th...,La découverte du spectre de la lumière blanche...
7,The sky of the first inhabitants A contemporar...,Le ciel des premiers habitants La vision conte...
8,Cartoon,Bande dessinée
9,Links,Liens


In [24]:
# helper function to convert the pos tag format into something compatible with the lemmatizer
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# turn text into clean tokens
def clean_data(doc, expand, lemma):
    #print(doc)
    if expand:
        doc = contractions.fix(doc)
    # remove every char that is not alphanumeric or end of sentence punctuation, keep spaces
    doc = doc.lower()
    doc = re.sub(r'[^ùûüÿàâæçéèêëïîôœÙÛÜŸÀÂÆÇÉÈÊËÏÎÔŒa-z0-9.!?]+', ' ', doc)
    tokens = wordpunct_tokenize(doc)
    #lowercase_tokens = [token.lower() for token in tokens]
    if lemma:
        pos = pos_tag(tokens)
        clean_tokens = [lemmatizer.lemmatize(
            word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in pos]
    else:
        clean_tokens = tokens
    return clean_tokens

In [25]:
def clean(df):
    df['en'] = df['en'].apply(lambda x: clean_data(x, expand=True, lemma=True))
    df['fr'] = df['fr'].apply(lambda x: clean_data(x, expand=False, lemma=False))
    return df

In [26]:
def split(df):
    # Split 80-10-10
    X_train, X_val_test, y_train, y_val_test = train_test_split(
            df["en"], df["fr"], test_size=0.2, random_state=seed)
    X_test, X_val, y_test, y_val = train_test_split(
            X_val_test, y_val_test, test_size=0.5, random_state=seed)
    return X_train, y_train, X_val, y_val, X_test, y_test

In [27]:
def create_tokenizer(text, max_words=0):
    # tokenizer = Tokenizer(num_words=max_words)
    if max_words == 0:
        tokenizer = Tokenizer()
    else:
        tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(text)
    return tokenizer


def encode_sequences(tokenizer, text, pad_len):
    seq = tokenizer.texts_to_sequences(text)
    seq = pad_sequences(seq, maxlen=pad_len, padding='post')
    return seq


def get_encodings(X, y=None, is_train=False, sc_tokenizer=None, tg_tokenizer=None, maxlen=40):
    # Only create and fit a new tokenizer on the training set
    if is_train:
        sc_tokenizer = create_tokenizer(X)
        tg_tokenizer = create_tokenizer(y)

    X_encoded = encode_sequences(sc_tokenizer, X, maxlen)
    y_encoded = encode_sequences(tg_tokenizer, y, maxlen)

    return X_encoded, y_encoded, sc_tokenizer, tg_tokenizer

In [28]:
def preprocess(df, is_clean):
    if not is_clean:
        df = df.dropna()
        df_clean = clean(df)
        df_clean = df_clean.dropna()
        df_clean.to_pickle('../fr-clean-data20.pkl')
    else:
        df_clean = pd.read_pickle('../fr-clean-data20.pkl')
        
    X_train, y_train, X_val, y_val, X_test, y_test = split(df_clean)
    return X_train, y_train, X_val, y_val, X_test, y_test

In [29]:
X_train, y_train, X_val, y_val, X_test, y_test = preprocess(df.head(20_000), is_clean=False)

In [30]:
# Turn sentences into tokenized and padded sequences
X_train_encoded, y_train_encoded, en_tokenizer, fr_tokenizer = get_encodings(X_train, y_train, is_train=True)

X_val_encoded, y_val_encoded, _, _ = get_encodings(X=X_val, is_train=False, y=y_val, sc_tokenizer=en_tokenizer, tg_tokenizer=fr_tokenizer)

X_test_encoded, y_test_encoded, _, _ = get_encodings(X_test, y_test, is_train=False, sc_tokenizer=en_tokenizer, tg_tokenizer=fr_tokenizer)

with open('../fr_train_data20.npy', 'wb') as f:
            np.save(f, X_train_encoded)
            np.save(f, y_train_encoded)

with open('../fr_test_data20.npy', 'wb') as f:
            np.save(f, X_test_encoded)
            np.save(f, y_test_encoded)

with open('../fr_valid_data20.npy', 'wb') as f:
            np.save(f, X_val_encoded)
            np.save(f, y_val_encoded)

In [31]:
print(len(en_tokenizer.word_index) + 1)
print(len(fr_tokenizer.word_index) + 1)

42577
57592


In [32]:
from keras.layers import LSTM, Embedding, Bidirectional, Dense, RepeatVector, TimeDistributed
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

def define_model(in_vocab_size, out_vocab_size, embedding_matrix=None, in_seq_length=40, out_seq_length=40, embedding_size=50):
    model = Sequential()
    model.add(Embedding(input_dim=in_vocab_size,
              output_dim=embedding_size, input_length=in_seq_length))
    # Encoder
    model.add(Bidirectional(LSTM(128)))
    # Decoder
    model.add(RepeatVector(out_seq_length))
    model.add(LSTM(256, return_sequences=True))
    # Prediction
    model.add(TimeDistributed(Dense(out_vocab_size, activation='softmax')))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(learning_rate=0.003),
                  metrics="accuracy")
    model.summary()
    return model

In [33]:
def train(model, X_train, y_train, X_val, y_val):
    history = model.fit(X_train, y_train, batch_size=128, epochs=1, validation_data=(X_val, y_val),
              verbose=1,
              callbacks=[
        EarlyStopping(
            monitor='val_loss',
            patience=2,
            restore_best_weights=True
        )
    ])
    model.save('../fr-model.h5')
    return history

In [36]:
en_vocab_size = len(en_tokenizer.word_index) + 1
fr_vocab_size = len(fr_tokenizer.word_index) + 1
print(en_vocab_size, fr_vocab_size)
#model = define_model(en_vocab_size, fr_vocab_size)

16195 23101


In [37]:
history = train(model, X_train_encoded, y_train_encoded, X_val_encoded, y_val_encoded)

2023-06-15 11:30:43.715891: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-06-15 11:30:43.940027: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-15 11:30:43.940737: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-15 11:30:43.941500: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG IN

125/125 [==============================] - ETA: 0s - loss: 5.1085 - accuracy: 0.4485

2023-06-15 11:33:09.802617: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-15 11:33:09.803513: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-15 11:33:09.804132: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

125/125 [==============================] - 155s 1s/step - loss: 5.1085 - accuracy: 0.4485 - val_loss: 4.2048 - val_accuracy: 0.4656


In [15]:
def vector_to_word(embedding, tokenizer):
    idx = np.argmax(embedding)
    print(idx)
    for word, transform in tokenizer.word_index.items():
        if transform == idx:
            return word
    return None


def get_sentences(sequences, tokenizer):
    predictions = []
    for sentence in sequences:
        predict = ''
        for emb in sentence:
            word = vector_to_word(emb, tokenizer)
            if word is not None:
                predict += word + ' '
        predictions.append(predict)
    return predictions

from keras.models import load_model
model = load_model("../fr-model-csanad.h5")
prediction = model.predict(X_train_encoded[0:100])
prediction_sentences = get_sentences(prediction, fr_tokenizer)
candidate_translations = [[sentence] for sentence in prediction_sentences]
print(candidate_translations)

2023-06-15 14:45:11.683439: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-15 14:45:11.685260: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-15 14:45:11.685866: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

4/4 [==============================] - 1s 88ms/step
4
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
767
767
1377
1377
1377
1377
1377
2692
2692
2692
2692
2692
2692
2692
2692
2692
2692
2692
2692
2692
2692
2692
2692
2692
2692
2692
2692
2692
2692
2692
2692
2692
2692
2692
2692
2692
2692
2692
2692
2692
4
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
1
1
0
0
0
0
0
0
0
0
0
0
0
0


In [42]:
from keras.utils.vis_utils import plot_model
#model = define_model(en_vocab_size, fr_vocab_size)
plot_model(model, to_file='../model_plot.png', show_shapes=True, show_layer_names=True)

16195 23101
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [125]:
from pickle import dump

dump(fr_tokenizer, open("../fr_tokenizer.pkl", "wb"))